In [1]:
import pandas as pd
from datetime import datetime
import re

In [2]:
# read in basline dates
df = pd.read_csv("HealthRhythmsAUDCTSI-BaselineDates_DATA_2023-07-27_1631.csv")
df

,record_id,redcap_event_name,visit_date,eligible_status
0,CF017AUD,baseline_arm_3,2021-11-19,1
1,CF021AUD,baseline_arm_3,2021-11-22,1
2,CF026AUD,baseline_arm_3,2021-11-30,1
3,CF037AUD,baseline_arm_3,2021-12-14,1
4,CF041AUD,baseline_arm_3,2021-12-07,1
...,...,...,...,...
137,MM312AUD,baseline_arm_3,2023-04-23,1
138,MM315AUD,baseline_arm_3,2023-04-29,1
139,MM318AUD,baseline_arm_3,2023-05-02,1
140,MM321AUD,baseline_arm_3,2023-05-17,1


In [3]:
# list nonvalid pts
nonvalid = ['CF270AUD','CF285AUD','CF286AUD','CM229AUD','CM236AUD','CM237AUD','CM251AUD','CM256AUD','CM271AUD','CM287AUD','CM301AUD','MF246AUD','MF249AUD','MF272AUD','MM216AUD','MM250AUD','MM260AUD','MM282AUD','MM293AUD']

In [4]:
# list withdrawn pts and withdrawn dates
withdrawn = pd.DataFrame({'id':['CF026AUD','CF231AUD','CF241AUD','CF276AUD','CF295AUD','CM263AUD','CM297AUD','MF258AUD','MF281AUD','MM028AUD','MM132AUD','MM133AUD','MM145AUD','MM232AUD','MM268AUD','MM290AUD','MM291AUD'],
             'withdraw_date':['1/21/2022','3/20/2021','3/20/2021','3/23/2023','3/31/2023','3/20/2023','3/20/2023','3/20/2023','3/31/2023','3/24/2022','8/20/2022','8/20/2022','7/28/2022','3/8/2023','4/17/2023','3/20/2021','1/19/2023']})
withdrawn['withdraw_date'] = withdrawn['withdraw_date'].apply(lambda x: pd.to_datetime(x, format="%m/%d/%Y"))
withdrawn

,id,withdraw_date
0,CF026AUD,2022-01-21
1,CF231AUD,2021-03-20
2,CF241AUD,2021-03-20
3,CF276AUD,2023-03-23
4,CF295AUD,2023-03-31
5,CM263AUD,2023-03-20
6,CM297AUD,2023-03-20
7,MF258AUD,2023-03-20
8,MF281AUD,2023-03-31
9,MM028AUD,2022-03-24


In [5]:
# exclude nonvalid
df = df.loc[~df['record_id'].isin(nonvalid)]
# cleanup
df['visit_date'] = df['visit_date'].apply(lambda x: pd.to_datetime(x))
df.drop(columns=['eligible_status'], inplace=True)
df['redcap_event_name'] = df['redcap_event_name'].apply(lambda x: re.split('_arm_3', x)[0])
df.rename(columns={'redcap_event_name':'event_name','visit_date':'event_date'},inplace=True)
df

/var/folders/71/p0nsprlx033b24hgn0jyt5n40000gp/T/ipykernel_34078/1428519538.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['visit_date'] = df['visit_date'].apply(lambda x: pd.to_datetime(x))
/var/folders/71/p0nsprlx033b24hgn0jyt5n40000gp/T/ipykernel_34078/1428519538.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['eligible_status'], inplace=True)
/var/folders/71/p0nsprlx033b24hgn0jyt5n40000gp/T/ipykernel_34078/1428519538.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,record_id,event_name,event_date
0,CF017AUD,baseline,2021-11-19
1,CF021AUD,baseline,2021-11-22
2,CF026AUD,baseline,2021-11-30
3,CF037AUD,baseline,2021-12-14
4,CF041AUD,baseline,2021-12-07
...,...,...,...
137,MM312AUD,baseline,2023-04-23
138,MM315AUD,baseline,2023-04-29
139,MM318AUD,baseline,2023-05-02
140,MM321AUD,baseline,2023-05-17


In [6]:
# calculate dates for each event and add them to the dataframe
for i, row in df.iterrows():
    events = pd.DataFrame({'record_id':row['record_id'],'event_name':['week_1','week_2','week_3','week_4','week_5','week_6','week_7','week_8','week_9','week_10','week_11','week_12','week_13','week_14'],
              'event_date':[row['event_date']+pd.Timedelta(days=7),row['event_date']+pd.Timedelta(days=14),row['event_date']+pd.Timedelta(days=21),row['event_date']+pd.Timedelta(days=28),
                            row['event_date']+pd.Timedelta(days=35),row['event_date']+pd.Timedelta(days=42),row['event_date']+pd.Timedelta(days=49),row['event_date']+pd.Timedelta(days=56),
                            row['event_date']+pd.Timedelta(days=63),row['event_date']+pd.Timedelta(days=70),row['event_date']+pd.Timedelta(days=77),row['event_date']+pd.Timedelta(days=84),
                            row['event_date']+pd.Timedelta(days=91),row['event_date']+pd.Timedelta(days=98)]})
    df = pd.concat([df,events])

df = df.sort_values(by=['record_id','event_date']).reset_index().drop(columns='index')
df_all = df.copy()

In [7]:
df_all['event_name'].value_counts()

baseline    123
week_1      123
week_2      123
week_3      123
week_4      123
week_5      123
week_6      123
week_7      123
week_8      123
week_9      123
week_10     123
week_11     123
week_12     123
week_13     123
week_14     123
Name: event_name, dtype: int64

In [8]:
df_all.to_csv('all_events.csv',index=False)

In [9]:
all_df_pre_20230206 = df_all.loc[df_all['event_date']<datetime(2023, 2, 6)]
all_df_pre_20230206.to_csv("all_events_pre_20230206.csv",index=False)

In [10]:
all_df_pre_20230206['event_name'].value_counts()

baseline    108
week_1      104
week_2      101
week_3       97
week_4       95
week_5       95
week_6       92
week_7       90
week_8       90
week_9       90
week_10      89
week_11      88
week_12      87
week_13      86
week_14      86
Name: event_name, dtype: int64

In [11]:
all_df_post_20230206 = df_all.loc[df_all['event_date']>=datetime(2023, 2, 6)]
all_df_post_20230206.to_csv("all_events_post_20230206.csv",index=False)

In [12]:
# remove events for for which pts were withdrawn
for i, row_i in df.iterrows():
    for j, row_j in withdrawn.iterrows():
        if row_i['record_id'] == row_j['id'] and row_i['event_date'] > row_j['withdraw_date']:
            df.drop(i,inplace=True)

df

,record_id,event_name,event_date
0,CF017AUD,baseline,2021-11-19
1,CF017AUD,week_1,2021-11-26
2,CF017AUD,week_2,2021-12-03
3,CF017AUD,week_3,2021-12-10
4,CF017AUD,week_4,2021-12-17
...,...,...,...
1840,PM142AUD,week_10,2022-09-13
1841,PM142AUD,week_11,2022-09-20
1842,PM142AUD,week_12,2022-09-27
1843,PM142AUD,week_13,2022-10-04


In [13]:
df.to_csv('events.csv',index=False)

In [14]:
df_pre_20230206 = df.loc[df['event_date']<datetime(2023, 2, 6)]
df_pre_20230206.to_csv("events_pre_20230206.csv",index=False)

In [15]:
df_post_20230206 = df.loc[df['event_date']>=datetime(2023, 2, 6)]
df_post_20230206.to_csv("events_post_20230206.csv",index=False)